<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/bugfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata
import ipywidgets as widgets
import datetime
engine = create_engine(userdata.get('supabase'), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [40]:
start_date = widgets.DatePicker(description='Start Date', disabled=False, value = datetime.date(2021,9,1))
end_date = widgets.DatePicker(description='End Date', disabled=False, value = datetime.date(2021,9,21))
limit = widgets.IntText(description='Limit', value=100, )

In [41]:
display(start_date, end_date, limit)

DatePicker(value=datetime.date(2021, 9, 1), description='Start Date')

DatePicker(value=datetime.date(2021, 9, 21), description='End Date')

IntText(value=100, description='Limit')

In [59]:
SQL_TEMPLATE_QUERY_LINECHART = f"""
WITH t AS (
    SELECT
        query,
        ROW_NUMBER() OVER (PARTITION BY date_part('hour', ts) ORDER BY COUNT(*) DESC) AS rnb
    FROM
        vizro.yandex_data yd
    WHERE
        ts BETWEEN '{start_date.value.strftime('%Y-%m-%d')}' AND '{end_date.value.strftime('%Y-%m-%d')}'
    GROUP BY query, date_part('hour', ts)
)
SELECT
    platform,
    date_part('hour', ts)::INT AS hour,
    query,
    COUNT(*)
FROM
    vizro.yandex_data yd
WHERE
    query IN (SELECT DISTINCT query FROM t WHERE rnb < 6)
GROUP BY
    platform, date_part('hour', ts), query
    {f'LIMIT {limit.value}' if limit.value else ''}
"""

In [60]:
select(SQL_TEMPLATE_QUERY_LINECHART)

,platform,hour,query,count
0,desktop,0,porno,14
1,desktop,0,xxx,8
2,desktop,0,айфон 13,8
3,desktop,0,доброе утро,1
4,desktop,0,календарь 2021,23
...,...,...,...,...
95,desktop,7,порно,30
96,desktop,7,с днем рождения,31
97,desktop,7,с днем рождения женщине,42
98,desktop,7,с днем рождения мужчине,27
